
# Training CIFAR10 classifier
Disclaimer: This notebook is an adopted version of [this tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).
<img src="static/cifar10.png" align="center"/>

This is it! You have seen how to define neural networks, compute loss and make updates to the weights of the network. Today we'll train CIFAR10 classifier in 2 modes:
1. train model from scratch
2. finetune pretrained model

But before start, let's understand what CIFAR10 dataset is.

**CIFAR10** - 10-class dataset of 32x32 RGB images. The main difference from MNIST is that in CIFAR10 images have 3 channels (RGB). CIFAR10 images are downscaled photos of **real** objects:  `airplane`, `automobile`, `bird`, `cat`, `deer`.
`dog`, `frog`, `horse`, `ship`, `truck`.

## Task 1 (2 points). Train model from scratch
We will do the following steps in order:

1. Load and normalize the CIFAR10 training and test datasets using
   ``torchvision``
2. Define a Convolutional Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the validation data



### 1. Loading and normalizing CIFAR10
Using ``torchvision``, it’s extremely easy to load CIFAR10.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

Parameters:

In [ ]:
batch_size = 4
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

CIFAR10 classes:

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

The output of torchvision datasets are PILImage images of range [0, 1]. We transform them to Tensors of normalized range ~[-1, 1]. This weird numbers are just means and stds calculated on ImageNet (check [this docs](https://pytorch.org/docs/master/torchvision/models.html) for more information). **Why should we normalize images?**

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

Setup dataloaders (downloading CIFAR10 can take some time):

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

print("len(train_dataset) =", len(train_dataset))

val_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)
val_dataloader= torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print("len(val_dataset) =", len(val_dataset))

Let us show some of the training images, for fun:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    img = img.cpu().numpy()
    plt.imshow(np.transpose(img, (1, 2, 0)))
    plt.show()


# get some random training images
images, labels = next(iter(train_dataloader))

print("images.shape =", images.shape)

print("labels =", labels)
print("labels.shape =", labels.shape)

# show images
imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

### Define a Convolutional Neural Network
Here we'll define convolutional neural network. Below you can find scheme of network. Implement it using PyTorch:

1. Conv (3 -> 6, 5x5, stride=1, padding=`same`)
2. ReLU
3. MaxPool (2x2, stride=2)
4. Conv (6 -> 16, 5x5, stride=1, padding=`same`)
5. ReLU
6. Linear (`???` -> 128)
7. ReLU
8. Linear (128 -> 64)
9. ReLU
10. Linear (64 -> 10)

Here padding=`same` means that the size of image doesn't change, so you have to calculate padding value yourself. Also you can see unknown `???` in 6th layer (calculate it yourself or find it emperically).

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class SimpleConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        ## define layers here
        
    def forward(self, x):
        
        ## code forward pass here
        
        return x


model = SimpleConvNet().to(device)

### 3. Define a Loss function and optimizer
Let's use a Classification Cross-Entropy loss and SGD with momentum.



In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

### 4. Train the network

This is when things start to get interesting. We simply have to loop over our dataloader, and feed the inputs to the
network and optimize.

In [ ]:
from tqdm import tqdm_notebook as tqdm

for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for batch in tqdm(train_dataloader):
        # get the inputs
        image_batch, label_batch = batch
        image_batch, label_batch = image_batch.to(device), label_batch.to(device)
        
        # zero the parameter gradients
        opt.zero_grad()

        # forward + backward + optimize
        outputs = ## your code here
        loss = ## your code here
        
        loss.backward()
        opt.step()
        
        running_loss += loss.item()
        
        
    print("[epoch {}] loss: {:.3}".format(epoch, running_loss / len(train_dataloader)))

### 5. Test the network on the validation data

We have trained the network for 2 epochs over the training dataset. But we need to check if the network has learnt anything at all.
We will check this by predicting the class label that the neural network outputs, and checking it against the ground-truth. If the prediction is correct, we add the sample to the list of correct predictions.

Okay, first step. Let us display an image from the validation set to get familiar.

In [ ]:
image_batch, label_batch = next(iter(val_dataloader))
image_batch, label_batch = image_batch.to(device), label_batch.to(device)

# print images
imshow(torchvision.utils.make_grid(image_batch))
print('GroundTruth: ', ' '.join('%5s' % classes[label_batch[j]] for j in range(batch_size)))

Okay, now let us see what the neural network thinks these examples above are:



In [ ]:
outputs = model(image_batch)

The outputs are energies for the 10 classes.
The higher the energy for a class, the more the network
thinks that the image is of the particular class.
So, let's get the index of the highest energy:



In [ ]:
predicted = torch.argmax(outputs, dim=1)

print("Predicted: {}".format(" ".join(classes[predicted[j]]for j in range(batch_size))))

The results seem pretty good.

Let us look at how the network performs on the whole dataset.



In [ ]:
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for batch in tqdm(val_dataloader):
        image_batch, label_batch = batch
        image_batch, label_batch = image_batch.to(device), label_batch.to(device)
        
        outputs = ## your code here
        predicted = ## your code here
        
        total += labels.size(0)
        correct += (predicted == label_batch).sum().item()

print("Accuracy of the network on the 10000 val images: {:.4}%".format(100 * correct / total))

That looks waaay better than chance, which is 10% accuracy (randomly picking a class out of 10 classes). Seems like the network learnt something. You're awesome.

## Task 2 (2 points). Finetune pretrained model

As we already know, **deep learning** is about hierarchical feature learning. It means that major part of neural network just learns how to extract features, and only last layers laearn how to solve target task (e.g. classification). So if we take some modern model carefully trained on a big dataset, it's likely that it has learnt to extract some useful features from data.

In this part of the seminar, we'll finetune [AlexNet](https://arxiv.org/abs/1404.5997) (one of the first deep CNN-architectures) trained on ImageNet (1000-class image dataset).

We'll do following steps:
1. Load and initialize the pretrained model
2. Freeze part of the network responsible for extracting *features*
3. Replace existing *classifying* part of the network with yours
4. Finetune resulting model

As you can read in the [AlexNet paper](https://arxiv.org/abs/1404.5997), takes images of size 224x224. Add proper resize to our transforms:

In [ ]:
transform = transforms.Compose([
    ## <- add resize here
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

Setup dataloaders (the same as before, but with different transform):

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

print("len(train_dataset) =", len(train_dataset))

val_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)
val_dataloader= torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print("len(val_dataset) =", len(val_dataset))

### 1. Load and initialize the pretrained model

Now let's load already pretrained AlexNet model. Luckily `torchvision` gives us easy interface to load popular pretrained models (weights downloading can take some time): 

In [ ]:
model = torchvision.models.alexnet(pretrained=True).to(device)

Look at [source code](https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py) for AlexNet for deeper understanding of how model works. Let's print it:

In [ ]:
print(model)

As we can see, AlexNet has 2 main submodules:
 - **features**: extracts high-abstract features from input images
 - **classifier**: classifies resulting features in 1000 ImageNet classes
 
`features` module is very useful for us, and we'll use it as it is. `classifier` module is more specific for ImageNet classes and we'll replace it with our own.

We don't want to finetune `features` module, so let's freeze it by setting `requiers_grad=False` to its parameters:

In [ ]:
## iterate over parameters and freeze them

Now let's build our own classifier. Use [nn.Sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential) for simplisity. Also try using [nn.Dropout](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout) just as in original model:

In [ ]:
classifier = ## your code here

Replace existing classifier with ours:

In [ ]:
model.classifier = classifier

Setup criterion and optimizer.

**Note**: we pass to optimizer only those parameters that we want to optimize.

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)

Run training for only 1 epoch:

In [ ]:
## you code here (you can just copy-paste from code above)

Validate:

In [ ]:
## you code here (you can just copy-paste from code above)

WOW! We seriously improved accuracy of our model by just finetuning several linear layers. Here is the power of finetuning!